优化算法:使用Keras提供的其他优化器，如梯度下降，看在其他算法下模型参数对模型训练和过拟合的速度有怎样的影响。
损失函数:尝试使用Keras其他可用的损失函数，探究选用其他的损失函数是否可以提升模型的性能。
学习率与迭代次数更新策略
更大的Batch Size:使用更大的Batch Size意味着模型在训练集和测试集上的数据操作规模更大了

In [1]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import *
#from keras.layers import Input
from keras.models import *
#from keras.models import Model
#from keras.layers import Input, Dense
#a = Input(shape=(32,))
#b = Dense(32)(a)
#model = Model(inputs=a, outputs=b)
from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers
from keras.applications import *

plt.ion()   # interactive mode 画图不阻止程序运行
#from keras.utils import multi_gpu_model 
#from keras.applications import ResNet50
#from keras.applications import VGG16
#from keras.applications import VGG19
#from keras.applications import Xception # TensorFlow ONLY
#from keras.applications import InceptionResNetV2
#from keras.applications import InceptionV3

#tf.keras.applications.inception_v3.InceptionV3
#tf.keras.applications.inception_resnet_v2.InceptionResNetV2
####################################################################
#设置GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

#设置项
#看具体的模型参数设置在:https://www.tensorflow.org/api_docs/python/tf/keras/applications
MODELS = {"InceptionResNetV2":InceptionResNetV2}
#MODELS = {"DenseNet121":DenseNet121,"DenseNet201":DenseNet201,"InceptionResNetV2":InceptionResNetV2}
#"InceptionV3":InceptionV3,"DenseNet121":DenseNet121,
 #       "DenseNet169":DenseNet169,"DenseNet201":DenseNet201,"Xception":Xception, 
 #       "InceptionResNetV2":InceptionResNetV2,
#"ResNet50":ResNet50, 
#"VGG16":VGG16,"VGG16":VGG19,"NASNetMobile":NASNetMobile

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels', 
#             'sleeve_length_labels', 'neck_design_labels', 'lapel_design_labels', 
#             'pant_length_labels','coat_length_labels']   
classes = ['pant_length_labels']    
fai_result = []

Using TensorFlow backend.


In [2]:
for KEY, MODLE in MODELS.items():
    #
    #为299*299,设置如下
    ppreprocess = preprocess_input
    if KEY in ["InceptionV3","Xception", "InceptionResNetV2"]:
        width = 299
    elif KEY == "NASNetLarge":
        width = 331
    else:
        width = 224
        ppreprocess = imagenet_utils.preprocess_input 
    print('######################在{0}下训练8个分类器####################'.format(KEY))
    for cur_class in classes:
        print('#######{0}:{1}####################'.format(KEY,cur_class ))
        

        #checkpointer = ModelCheckpoint(filepath='../models/{0}/{0}_{1}效果很好.best.h5'.format(prefix_cls,KEY), verbose=1, 
        #                            save_best_only=True)
        pre_attr = cur_class.split('_')[0]
        model=load_model('../models/{0}/{0}_{1}效果很好.best.h5'.format(pre_attr, KEY),
            {'imagenet_utils':imagenet_utils})######当时在这不加这句dict就会出现imagenet_utils未定义,貌似是Lamda层的影响
        #解决办法:https://github.com/keras-team/keras/issues/4609,到处第3个,最下面有:
        
        print("模型装在完毕,开始测试!")
        #测试集上预测并输出结果
        df_test = pd.read_csv('../test/z_rank/Tests/question.csv', header=None)
        df_test.columns = ['image_id', 'class', 'x']
        del df_test['x']
        
        df_load = df_test[(df_test['class'] == cur_class)].copy()
        df_load.reset_index(inplace=True)
        del df_load['index']
        
        n = len(df_load)
        X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

        for i in range(n):
            X_test[i] = cv2.resize(cv2.imread('../test/z_rank/{0}'.format(df_load['image_id'][i])), (width, width))
        test_np = model.predict(X_test, batch_size=256)
        result = []
        for i, row in df_load.iterrows():
            tmp_list = test_np[i]
            tmp_result = ''
            for tmp_ret in tmp_list:
                tmp_result += '{:.4f};'.format(tmp_ret)

            result.append(tmp_result[:-1])

        df_load['result'] = result     
        df_load.to_csv('../result/B/B_{0}_{1}.csv'.format(pre_attr, KEY), header=None, index=False)

print("completed!")

######################在InceptionResNetV2下训练8个分类器####################
#######InceptionResNetV2:pant_length_labels####################
模型装在完毕,开始测试!
completed!
